In [ ]:
#Se importa librería de Visión Computacional abierta
import cv2
#Se importa librería de cálculos numéricos optimizados
import numpy as np
#Se importa la función cv_imshow para mostrar imágenes en Google Colab
from google.colab.patches import cv2_imshow
#Se importe ndimage para cálculos científicos con imágenes
from scipy import ndimage
#Se importa función peak_local_max para encontrar picos locales máximos
from skimage.feature import peak_local_max
#Se importa watershed para aplicar el algoritmo de cuenca
from skimage.morphology import watershed
#Se importa la librería de visualización de datos
import matplotlib.pyplot as plt
#Se importa la librería de manejo de imágenes
import imutils

In [ ]:
#Se importa la imagen sobre la cual queremos contar objetos y calcular su área
ImagenBGR = cv2.imread('/content/peje-coins.jpg')
#Se muestra la imagen cargada en el paso anterior
cv2_imshow(ImagenBGR)

In [ ]:
#Definimos una función, la cual sólo pedira una imagen entrada sobre la cual se contarán objetos y se calculará el área simultaneamente
def ContadorYCalculador_de(ImagenBGR):
    #Se crea una matriz que funcionará como kernel para la erosión y dilatación    
    kernel = np.ones((5,5), np.uint8)
    #La imagen cargada dentro de la función pasará a ser escala de grises
    ImagenGray = cv2.cvtColor(ImagenBGR, cv2.COLOR_BGR2GRAY)
    #La imagen en escala de grises, se le aplicarán dos filtros de difuminado Gaussiano, para quitar el brillo a las monedas (ruido)
    ImagenGaussiana = cv2.GaussianBlur(ImagenGray,(3,3),0)
    #Se le aplica otro difuminado Gaussiano por si hubiera todavía ruido
    ImagenGaussiana = cv2.GaussianBlur(ImagenGaussiana,(3,3),0)
    #Se binariza la imagen después de los difuminados, el rango a ser negro (0) será del 240 a 255, el resto será blanco (1)
    Umbral, ImagenBinarizada = cv2.threshold(ImagenGaussiana, 240, 255, cv2.THRESH_BINARY_INV)
    #Se erosiona la imagen, en caso de haber unión entre las monedas
    ImagenErosionada = cv2.erode(ImagenBinarizada, kernel, iterations=3)
    #Se dilata la imagen para volver casi al estado original, solo que sin unión entre las monedas
    ImagenDilatada = cv2.dilate(ImagenErosionada, kernel, iterations=2)
    #Se calcula la distancia euclidiana, para determinar los centro de mayor intesidad y segmentar mejor las monedas 
    DistanciaEuclidean  = ndimage.distance_transform_edt(ImagenDilatada)
    #plt.imshow(DistanciaEuclidean) #para ver el mapa de calor generado 
    #Se calculan los picos locales máximos, para tratar al mapa de calor, como relieves montañosas
    PicosLocalesMaximos = peak_local_max(DistanciaEuclidean, indices=False, min_distance=20, labels=ImagenDilatada)
    #Se encuentran las marcas encontradas por los picos locales máximos y una estructura, sólo se agarra el primer valor por eso al final se pone "[0]""
    Marcas = ndimage.label(PicosLocalesMaximos, structure=np.ones((3,3)))[0]
    #Las etiquetas se encuentran por medio del algortimo de cuenca, de ahí se segmentarán las máscaras correctamente con respecto a una base (ImagenDilatada es la base)
    #La distancia euclidiana debe ser negativa, por eso el signo (-)
    Etiquetas = watershed(-DistanciaEuclidean, Marcas, mask=ImagenDilatada)
    #Se nombra como variable global (para no tener errores después)
    global CantidadDeDinero
    #Siempre que se ingrese una imagen o nuevas imágenes, el conteo comenzará desde 0
    CantidadDeDinero = 0
    #Por cada valor de la variable Etiquetas, se determinará si está registrado el fondo o la región de interés (la máscara)
    for Etiqueta in np.unique(Etiquetas):
        #Si la región de interés es 0, entonces es el fondo, porque lo que "continue" hará que ese valor no se tome en cuenta
        if Etiqueta == 0:
            continue
        #Se crea una máscara totalmente oscuro o un arreglo de solo (0)
        Mascara = np.zeros(ImagenGray.shape, dtype="uint8")
        #Si el resultado de esa comparativa, es igual, quiere decir que se trata de la región de interés, así que se le asigna un 255 o (1)--> región blanca
        Mascara[Etiquetas == Etiqueta] = 255
        #Para encontrar los contornos externos de cada máscara (monedas) se usa la siguiente función
        Contornos = cv2.findContours(Mascara.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #Se toman los contornos mediante la función de imutils
        Contornos = imutils.grab_contours(Contornos)
        #Se encuentra el número de objetos con este cálculou usando len() y restandole 1 (índice) al resultado
        CantidadDeMonedas = len(np.unique(Etiquetas))-1

        #Se analizará cada contorno (moneda) 
        for Contorno in Contornos:
            #Se calcula el área de la moneda
            area = cv2.contourArea(Contorno)
            #Se calcula un círculo promedio equivalente a la forma de su contorno, para encontrar su centro (y radio de pasada xd)
            ((x,y), r) = cv2.minEnclosingCircle(Contorno)
            #Se muestra el área en formato string en la imagen, ésto se hace con el fin de ver calcular los umbrales para cada moneda y su respectivo valor
            cv2.putText(ImagenBGR,str(area), (int(x) -20, int(y)-25), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1 )
            #Una vez visto los valores del área de cada moneda, se procede a determinar sobre qué intérvalos se encuentra el área de cada moneda con su valor
            #Ejemplo: La moneda de 5 peso, dio como resultado 3254.5 y 3320.9 por lo que los rangos entre los que se encuentran estos valores son 
            #Min: 3200 y Max: 3400
            if area>1100 and area<1200:
                #La variable cantidad de dinero, irá sumando el valor de la moneda, de acuerdo al valor del área del contorno encontrado
                CantidadDeDinero += 0.05
            elif area>1300 and area<1450:
                CantidadDeDinero += 0.1
            elif area>1600 and area<1900:
                CantidadDeDinero += 0.2
            elif area>2300 and area<2450:
                CantidadDeDinero += 0.5
            elif area>2100 and area<2250:
                CantidadDeDinero += 1
            elif area>2500 and area<2900:
                CantidadDeDinero += 2
            elif area>3200 and area<3400:
                CantidadDeDinero += 5
            elif area>3800 and area<4200:
                CantidadDeDinero += 10
            elif area>5000 and area<5450:
                CantidadDeDinero += 20
            
    #cv2_imshow(ImagenBGR)
    #plt.imshow(DistanciaEuclidean)
    #print(CantidadDeMonedas)
    #print(CantidadDeDinero)
    
    #Se retornan los valores que serán resultado de utilizar esta función creada
    return CantidadDeMonedas, CantidadDeDinero

In [ ]:
#Se crean dos variables que guardarán los valores retornados por la función ContadorYCalculador_de()
#Dentro de los paréntesis siempre se deberá colocar una imagen, de preferencia en BGR
CantidadMonedas, CantidadDinero = ContadorYCalculador_de(ImagenBGR)
print("Número de monedas encontradas:",CantidadMonedas)
print("Cantidad de dinero calculado:", CantidadDinero)

Número de monedas encontradas: 18
Cantidad de dinero calculado: 77.7
